In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st
from sklearn.preprocessing import OrdinalEncoder
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [79]:
#read csv
df = pd.read_csv('Data/Levels_Fyi_Salary_Data.csv')

In [80]:
#standardize column names
df.columns = df.columns.str.lower()

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62642 entries, 0 to 62641
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                62642 non-null  object 
 1   company                  62637 non-null  object 
 2   level                    62523 non-null  object 
 3   title                    62642 non-null  object 
 4   totalyearlycompensation  62642 non-null  int64  
 5   location                 62642 non-null  object 
 6   yearsofexperience        62642 non-null  float64
 7   yearsatcompany           62642 non-null  float64
 8   tag                      61788 non-null  object 
 9   basesalary               62642 non-null  float64
 10  stockgrantvalue          62642 non-null  float64
 11  bonus                    62642 non-null  float64
 12  gender                   43102 non-null  object 
 13  otherdetails             40137 non-null  object 
 14  cityid                

In [82]:
#drop columns that I don't want 
df = df.drop(['timestamp', 'rownumber', 'tag', 'title', 'otherdetails', 'cityid', 'dmaid', 'masters_degree', 'bachelors_degree', 
              'doctorate_degree', 'highschool', 'some_college', 'race_asian','race_white', 
              'race_two_or_more', 'race_black', 'race_hispanic', 'race' ], axis=1)

In [54]:
#keep only total yearly compensation
df = df.drop(['basesalary', 'stockgrantvalue', 'bonus' ], axis=1)

In [55]:
# Clean the "Gender" column to keep only Male and Female
df = df.dropna(subset=['gender'])
df.reset_index(drop=True, inplace=True)
df.drop(df[df["gender"] == "Other"].index, inplace = True)
df.drop(df[df["gender"] == 'Title: Senior Software Engineer'].index, inplace = True)
df['gender'].value_counts(dropna = False)

Male      35702
Female     6999
Name: gender, dtype: int64

In [56]:
#new gender column is_female
def gender(val):
    if val == "Female":
        return 1
    else:
        return 0

In [57]:
#create the is_female column
df["is_female"] = df['gender'].apply(gender)
df['is_female'].value_counts()

0    35702
1     6999
Name: is_female, dtype: int64

In [58]:
#fill company NaN
df['company'] = df['company'].fillna("Unknown")

In [59]:
#transform company comumn into is_faang? assuming they might be paid more
def faanging(val):
    if val == 'Amazon' or val == 'Facebook' or val == 'Apple' or val == 'Netflix' or val == 'Google':
        return 1
    else:
        return 0

In [60]:
#create new column is FAANG? (assuming FAANG might pay more than other companies)
df['is_faang'] = df['company'].apply(faanging)
df["is_faang"].value_counts()

0    30609
1    12092
Name: is_faang, dtype: int64

In [61]:
#looking at title 
df['title'].value_counts()

Software Engineer               28724
Product Manager                  3038
Software Engineering Manager     2278
Data Scientist                   1718
Hardware Engineer                1512
Product Designer                 1126
Technical Program Manager         888
Solution Architect                797
Management Consultant             616
Business Analyst                  525
Marketing                         419
Mechanical Engineer               304
Sales                             279
Recruiter                         274
Human Resources                   203
Name: title, dtype: int64

In [62]:
#droping titles that aren't explicitely STEM
df.drop(df[(df['title'] == 'Marketing') | (df['title'] == 'Recruiter') | (df['title'] == 'Human Resources') 
           | (df['title'] == 'Sales') 
           | (df['title'] == 'Management Consultant')].index, inplace=True )

In [63]:
#looking at location
df['location'].value_counts()

Seattle, WA                                5595
San Francisco, CA                          3994
New York, NY                               2846
Redmond, WA                                1803
Sunnyvale, CA                              1503
Mountain View, CA                          1441
San Jose, CA                               1314
Austin, TX                                 1012
Menlo Park, CA                              983
Cupertino, CA                               955
Bangalore, KA, India                        915
Boston, MA                                  763
Santa Clara, CA                             757
Palo Alto, CA                               744
London, EN, United Kingdom                  712
San Diego, CA                               530
Bellevue, WA                                496
Chicago, IL                                 472
Toronto, ON, Canada                         444
Los Angeles, CA                             421
Vancouver, BC, Canada                   

In [64]:
#split the location in 3
df["country"] = df['location'].str.split(',').apply(lambda x: x[2].strip() if len(x)>2 else "USA")

In [65]:
df["country"].value_counts()

USA                     33882
India                    2170
Canada                   1196
United Kingdom            824
Germany                   416
Israel                    235
Singapore                 227
Ireland                   217
Taiwan                    216
Russia                    197
Netherlands               184
Australia                 162
Switzerland               139
Spain                      77
China                      68
Japan                      58
Poland                     57
Ukraine                    51
France                     51
Sweden                     47
Czech Republic             39
Brazil                     34
Luxembourg                 26
Indonesia                  22
Hong Kong (SAR)            21
Romania                    21
Hungary                    17
United Arab Emirates       17
Belarus                    16
Korea                      14
Thailand                   14
Argentina                  14
Colombia                   13
Nigeria   

In [66]:
df.drop(df[df['country'] != 'USA'].index, inplace=True) #keep the states only
df['state'] = df['location'].str.split(",").apply(lambda x: x[1].strip())
df.drop(df[df['state'] == 'Israel'].index, inplace=True)
df["state"].value_counts()

CA    14348
WA     8088
NY     2958
TX     1760
MA     1146
VA      633
IL      522
OR      457
CO      400
GA      348
DC      334
NC      324
PA      316
NJ      280
AZ      257
FL      193
MN      193
UT      168
MI      146
MO      138
OH      127
WI      108
MD       72
IN       68
CT       66
AR       63
TN       56
DE       44
KS       30
IA       24
ID       22
NH       22
LA       20
AL       19
NE       18
SC       18
NV       17
KY       16
RI       14
OK       11
WV       10
NM        7
MT        6
VT        5
MS        2
HI        2
ND        2
ME        2
WY        1
Name: state, dtype: int64

In [67]:
df["title"].value_counts()

Software Engineer               23402
Product Manager                  2681
Software Engineering Manager     1904
Data Scientist                   1468
Hardware Engineer                1339
Product Designer                 1002
Technical Program Manager         778
Solution Architect                602
Business Analyst                  428
Mechanical Engineer               277
Name: title, dtype: int64

In [68]:
df_title_to_dum = df["title"]
df_title_dum = pd.get_dummies(df_title_to_dum, drop_first = True)
#df_title_dum
#business analyst is being dropped

In [69]:
df =pd.concat([df, df_title_dum], axis=1 )
df.head(10)

,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,gender,education,is_female,is_faang,country,state,Data Scientist,Hardware Engineer,Mechanical Engineer,Product Designer,Product Manager,Software Engineer,Software Engineering Manager,Solution Architect,Technical Program Manager
0,GE Digital,Senior,Software Engineer,140000,"San Ramon, CA",4.0,4.0,Male,NaN,0,0,USA,CA,0,0,0,0,0,1,0,0,0
1,Amazon,L5,Software Engineer,172000,"Seattle, WA",12.0,2.0,Male,NaN,0,1,USA,WA,0,0,0,0,0,1,0,0,0
2,Uber,L5,Software Engineer,325000,"San Francisco, CA",7.0,2.0,Male,NaN,0,0,USA,CA,0,0,0,0,0,1,0,0,0
3,Microsoft,60,Software Engineer,150000,"Seattle, WA",4.0,0.0,Male,NaN,0,0,USA,WA,0,0,0,0,0,1,0,0,0
4,Google,L6,Software Engineer,640000,"Mountain View, CA",11.0,4.0,Male,NaN,0,1,USA,CA,0,0,0,0,0,1,0,0,0
5,Toyota Research Institute,L3,Software Engineer,189000,"Boston, MA",0.0,0.0,Male,NaN,0,0,USA,MA,0,0,0,0,0,1,0,0,0
6,Microsoft,62,Software Engineer,230000,"Bellevue, WA",3.0,0.0,Male,NaN,0,0,USA,WA,0,0,0,0,0,1,0,0,0
7,Oracle,IC-4,Software Engineer,128000,"Santa Clara, CA",9.0,3.0,Male,NaN,0,0,USA,CA,0,0,0,0,0,1,0,0,0
8,Apple,ICT 3,Software Engineer,176000,"Cupertino, CA",3.0,1.0,Female,NaN,1,1,USA,CA,0,0,0,0,0,1,0,0,0
9,SAP,L4,Software Engineer,205000,"Palo Alto, CA",10.0,0.0,Male,NaN,0,0,USA,CA,0,0,0,0,0,1,0,0,0


## until here, education hasn't been touched, still has NaN
#### 1st option: we drop the NaN and encode the education column. education is included in the model. 

In [70]:
df['education'].value_counts(dropna=False)

NaN                  13180
Master's Degree      10893
Bachelor's Degree     8083
PhD                   1280
Some College           265
Highschool             180
Name: education, dtype: int64

In [71]:
#let's drop the NaN
df = df.dropna(subset=['education'])
df.reset_index(drop=True, inplace=True)
df['education'].value_counts()

Master's Degree      10893
Bachelor's Degree     8083
PhD                   1280
Some College           265
Highschool             180
Name: education, dtype: int64

In [1]:
df.head(10)

NameError: name 'df' is not defined

In [72]:
#ordinal encoder of the education column
from sklearn.preprocessing import OrdinalEncoder
array_cat_ord = df["education"].values
array_cat_ord =array_cat_ord.reshape(-1,1)
array_cat_ord

array([['PhD'],
       ["Master's Degree"],
       ['PhD'],
       ...,
       ["Master's Degree"],
       ["Master's Degree"],
       ["Master's Degree"]], dtype=object)

In [73]:
encoder = OrdinalEncoder(categories=[["Highschool", 'Some College', 'Bachelor\'s Degree', 'Master\'s Degree', "PhD"]])
encoder.fit(array_cat_ord)
encoded_array = encoder.transform(array_cat_ord)
                                    

In [74]:
cat_ord_encoded = pd.DataFrame(encoded_array, columns=['education_enc'])
df = pd.concat([df, cat_ord_encoded], axis = 1)



In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20701 entries, 0 to 20700
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   company                       20701 non-null  object 
 1   level                         20665 non-null  object 
 2   title                         20701 non-null  object 
 3   totalyearlycompensation       20701 non-null  int64  
 4   location                      20701 non-null  object 
 5   yearsofexperience             20701 non-null  float64
 6   yearsatcompany                20701 non-null  float64
 7   gender                        20701 non-null  object 
 8   education                     20701 non-null  object 
 9   is_female                     20701 non-null  int64  
 10  is_faang                      20701 non-null  int64  
 11  country                       20701 non-null  object 
 12  state                         20701 non-null  object 
 13  D

In [76]:
df['title'].value_counts()

L4                                                    1781
L5                                                    1732
L3                                                    1146
L6                                                     916
Senior Software Engineer                               543
L2                                                     483
L1                                                     319
E5                                                     287
L7                                                     281
ICT4                                                   279
IC4                                                    261
61                                                     248
ICT3                                                   244
Senior                                                 243
Software Engineer                                      242
62                                                     242
IC3                                                    2

In [ ]:
#numerical df for modelling
#df_model_one = df.select_dtypes(include = np.number)
#df_model_one.info()

In [ ]:
#df_model_one.to_csv('df_model_one.csv', index=False)

### Model 2 - let's include location


In [ ]:
df.head(10)

In [28]:
#let's dummifying the states
df_state_to_dum = df["state"]
df_state_dum = pd.get_dummies(df_state_to_dum, drop_first = True)
df =pd.concat([df, df_state_dum], axis=1 )


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20701 entries, 0 to 20700
Data columns (total 70 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   company                       20701 non-null  object 
 1   title                         20701 non-null  object 
 2   totalyearlycompensation       20701 non-null  int64  
 3   location                      20701 non-null  object 
 4   yearsofexperience             20701 non-null  float64
 5   yearsatcompany                20701 non-null  float64
 6   gender                        20701 non-null  object 
 7   education                     20701 non-null  object 
 8   is_female                     20701 non-null  int64  
 9   is_faang                      20701 non-null  int64  
 10  country                       20701 non-null  object 
 11  state                         20701 non-null  object 
 12  Data Scientist                20701 non-null  uint8  
 13  H

In [31]:
#numerical df for modelling
df_model_two = df.select_dtypes(include = np.number)
df_model_two = df_model_two.dropna(subset=['totalyearlycompensation'])
df_model_two.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20701 entries, 0 to 20700
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   totalyearlycompensation       20701 non-null  int64  
 1   yearsofexperience             20701 non-null  float64
 2   yearsatcompany                20701 non-null  float64
 3   is_female                     20701 non-null  int64  
 4   is_faang                      20701 non-null  int64  
 5   Data Scientist                20701 non-null  uint8  
 6   Hardware Engineer             20701 non-null  uint8  
 7   Mechanical Engineer           20701 non-null  uint8  
 8   Product Designer              20701 non-null  uint8  
 9   Product Manager               20701 non-null  uint8  
 10  Software Engineer             20701 non-null  uint8  
 11  Software Engineering Manager  20701 non-null  uint8  
 12  Solution Architect            20701 non-null  uint8  
 13  T

In [32]:
df_model_two.to_csv('df_model_two.csv', index=False)

In [34]:
df_model_two["WV"].value_counts()

0    20694
1        7
Name: WV, dtype: int64